# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=15)

<IB connected to 127.0.0.1:4002 clientId=15>

### Streaming tick data

Create some Forex contracts:

In [3]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

Request streaming ticks for them:

In [4]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [5]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 9, 23, 9, 52, 43, 601018, tzinfo=datetime.timezone.utc), bid=1.09815, bidSize=5000000, ask=1.09816, askSize=2000000, prevBid=1.09814, prevBidSize=2000000, prevAsk=1.09815, prevAskSize=3000000, high=1.10255, low=1.0966, close=1.102, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [8]:
ticker.marketPrice()

1.098145

In [11]:
type(ticker)

ib_insync.ticker.Ticker

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [12]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,5000000,1.09818,1.09819,2000000,1.10255,1.0966,1.102
USDJPY,1000000,107.499,107.5,2000000,107.77,107.31,107.56
GBPUSD,1000000,1.24451,1.24455,2000000,1.2491,1.2423,1.2474
USDCHF,1000000,0.99101,0.99104,2000000,0.99225,0.9893,0.9909
USDCAD,1000000,1.32996,1.33,2000000,1.33055,1.32635,1.3263
AUDUSD,2000000,0.67711,0.67714,6500000,0.67815,0.6765,0.6766


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [13]:
for contract in contracts:
    ib.cancelMktData(contract)

### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [9]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 7, 11, 17, 33, 53, 662953, tzinfo=datetime.timezone.utc), bid=1.12479, bidSize=4000000, ask=1.12481, askSize=4000000, prevBid=1.12476, prevBidSize=2000000, prevAsk=1.12479, prevAskSize=12000000, high=1.12855, low=1.1245, close=1.1251, ticks=[], tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2019, 7, 11, 17, 33, 53, 662953, tzinfo=datetime.timezone.utc), bidPrice=1.12479, askPrice=1.12481, bidSize=4000000, askSize=4000000, tickAttribBidAsk=TickAttribBidAsk())], domBids=[], domAsks=[], domTicks=[])


### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [10]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2019, 7, 11, 17, 34, 18, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.1248, priceAsk=1.12481, sizeBid=2000000, sizeAsk=1000000)

In [11]:
ticks

[HistoricalTickBidAsk(time=datetime.datetime(2019, 7, 11, 17, 32, 27, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.12479, priceAsk=1.12481, sizeBid=2000000, sizeAsk=5000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 7, 11, 17, 32, 28, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.12479, priceAsk=1.12481, sizeBid=2000000, sizeAsk=4000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 7, 11, 17, 32, 28, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.1248, priceAsk=1.12481, sizeBid=2000000, sizeAsk=4000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 7, 11, 17, 32, 28, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.1248, priceAsk=1.12481, sizeBid=2000000, sizeAsk=5000000),
 HistoricalTickBidAsk(time=datetime.datetime(2019, 7, 11, 17, 32, 28, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.1248, priceAsk=1.

In [12]:
util.df(ticks)

,time,tickAttribBidAsk,priceBid,priceAsk,sizeBid,sizeAsk
0,2019-07-11 17:32:27+00:00,TickAttribBidAsk(),1.12479,1.12481,2000000,5000000
1,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12479,1.12481,2000000,4000000
2,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,2000000,4000000
3,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,2000000,5000000
4,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,2000000,6000000
5,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,2000000,5000000
6,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,2000000,4000000
7,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,3000000,3000000
8,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,3000000,2000000
9,2019-07-11 17:32:28+00:00,TickAttribBidAsk(),1.12480,1.12481,3000000,1000000


In [13]:
ib.disconnect()